In [2]:
import shapes
import numpy as np
import skimage.measure as measure
import skimage.transform as transform
import matplotlib.pyplot as plt

In [4]:
all_data = shapes.load_shapes("../shapes.h5", stim_class=None)
removed_images = [] # list of (image, reason)

In [5]:
# get the relevant subset of data we want to work with
data = all_data[all_data['stim_class'].isin(['SingleElectrode', 'MultiElectrode'])]
data = data[data['implant'] == 'ArgusII']

In [11]:
data.groupby(['stim_class', 'num_regions']).count()

subject  amp1  amp2  area  compactness  date  \
stim_class      num_regions                                                 
MultiElectrode  1               1831  1831  1831  1831         1831  1831   
                2                958   958   958   958          958   958   
                3                 64    64    64    64           64    64   
                4                  4     4     4     4            4     4   
SingleElectrode 1                623   623   623   623          623   623   
                2                 15    15    15    15           15    15   

                             eccentricity  electrode1  electrode2  experiment  \
stim_class      num_regions                                                     
MultiElectrode  1                    1831        1831        1831        1831   
                2                     958         958         958         958   
                3                      64          64          64          64   
                4                       4           4           4           4   
SingleElectrode 1                     623         623         623         623   
                2                      15          15          15          15   

                             filename  freq  image  implant  orientation  \
stim_class      num_regions                                                
MultiElectrode  1                1831  1831   1831     1831         1831   
                2                 958   958    958      958          958   
                3                  64    64     64       64           64   
                4                   4     4      4        4            4   
SingleElectrode 1                 623   623    623      623          623   
                2                  15    15     15       15           15   

                             pdur  trial  x_center  y_center  img_shape  
stim_class      num_regions                                              
MultiElectrode  1            1831   1831      1831      1831       1831  
                2             958    958       958       958        958  
                3              64     64        64        64         64  
                4               4      4         4         4          4  
SingleElectrode 1             623    623       623       623        623  
                2              15     15        15        15         15

In [14]:
data

,subject,amp1,amp2,area,compactness,date,eccentricity,electrode1,electrode2,experiment,...,image,implant,num_regions,orientation,pdur,stim_class,trial,x_center,y_center,img_shape
0,12-005,1.5,0.0,0,1.0,20091028,0.0,A02,,172407,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,1,0.0,0.45,MultiElectrode,9,256.0,192.0,"(384, 512)"
1,12-005,1.5,0.0,0,1.0,20091028,0.0,A02,,172407,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,1,0.0,0.45,MultiElectrode,17,256.0,192.0,"(384, 512)"
2,12-005,1.5,0.0,0,1.0,20091028,0.0,A02,,172407,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,1,0.0,0.45,MultiElectrode,30,256.0,192.0,"(384, 512)"
3,12-005,1.5,0.0,0,1.0,20091028,0.0,A02,,172407,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,1,0.0,0.45,MultiElectrode,38,256.0,192.0,"(384, 512)"
4,12-005,1.5,0.0,0,1.0,20091028,0.0,A02,,172407,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,1,0.0,0.45,MultiElectrode,51,256.0,192.0,"(384, 512)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3930,52-001,2.0,2.0,0,1.0,20100506,0.0,F04,F07,103606,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,2,0.0,0.45,MultiElectrode,2,256.0,192.0,"(384, 512)"
3931,52-001,2.0,2.0,0,1.0,20100506,0.0,F04,F07,103606,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,2,0.0,0.45,MultiElectrode,14,256.0,192.0,"(384, 512)"
3932,52-001,2.0,2.0,0,1.0,20100506,0.0,F04,F07,103606,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,2,0.0,0.45,MultiElectrode,23,256.0,192.0,"(384, 512)"
3933,52-001,2.0,2.0,0,1.0,20100506,0.0,F04,F07,103606,...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",ArgusII,2,0.0,0.45,MultiElectrode,37,256.0,192.0,"(384, 512)"


# TODO
Manually investigate all instances of 2 phosphene for single electrode, as well as >2 phophenes for multi electrode.
Also compare phosphene area to other phosphenes for the same experiment, and investigate any trials with outlier area.

When you find an image that needs to be removed or changed, remove or modify it from all_data, add it to removed images, along with a very brief description of why it was removed/changed

In [ ]:
all_data.remove(index)
removed_images.append((image, "small phosphene"))









In [ ]:
shapes.save_shapes(all_data, )